In [2]:
import pandas as pd 
import numpy as np

In [3]:
train_main = pd.read_csv("../data/task1/train_1.7.csv", encoding="cp1251")
test_main = pd.read_csv("../data/task1/test_1.9.csv", encoding="cp1251")

train_aux_coords = pd.read_csv("../data/task1_additional/coords_train_1.1.csv", encoding="cp1251")
test_aux_coords = pd.read_csv("../data/task1_additional/coords_train_1.1.csv", encoding="cp1251")

train_aux_frac = pd.read_csv("../data/task1_additional/frac_train_1.csv", encoding="cp1251")
test_aux_frac = pd.read_csv("../data/task1_additional/frac_test_1.csv", encoding="cp1251")

train_aux_gdis = pd.read_csv("../data/task1_additional/gdis_train1.2.csv", encoding="cp1251")
test_aux_gdis = pd.read_csv("../data/task1_additional/gdis_test1.2.csv", encoding="cp1251")

In [13]:
train_aux_coords.head()

,ПЛАСТ,УСТЬЕ_X,УСТЬЕ_Y,ПЛАСТ_X,ПЛАСТ_Y,Альтитуда,well_hash
0,93b03192e539de336932ea455adc9ec6,3.557805e+06,1.532290e+07,1.242640e+07,3.780937e+07,50.97,53514b4c4150ad897d82dd7d42cfc1a5
1,ac48576d40a88015267d1d2b4e2c134c,3.557805e+06,1.532290e+07,1.242640e+07,3.780937e+07,50.97,53514b4c4150ad897d82dd7d42cfc1a5
2,b77e6ff069ae1e32a78d687b59ac7703,3.557805e+06,1.532290e+07,1.242640e+07,3.780937e+07,50.97,53514b4c4150ad897d82dd7d42cfc1a5
3,ac48576d40a88015267d1d2b4e2c134c,3.561735e+06,1.532271e+07,1.243326e+07,3.781279e+07,47.41,fe017a547ac58cc7366a60cc6bcd3915
4,93b03192e539de336932ea455adc9ec6,3.562573e+06,1.532146e+07,1.243424e+07,3.781082e+07,52.23,8f3131afe66d3266c23142786786b265


In [34]:
train_main.head(12)

,Скважина,Дата,ГТМ,Метод,Характер работы,Состояние,"Время работы, ч",Время накопления,"Нефть, т","Попутный газ, м3",...,Характеристический дебит жидкости,Время в работе,Время в накоплении,ГП - Забойное давление,ГП(ИДН) Дебит жидкости,ГП(ИДН) Дебит жидкости скорр-ый,ГП(ИДН) Прирост дефита нефти,ГП(ГРП) Дебит жидкости,ГП(ГРП) Дебит жидкости скорр-ый,ГП - Общий прирост Qн
0,53514b4c4150ad897d82dd7d42cfc1a5,2012-11-30,0.0,ЭЦН,НЕФ,РАБ.,744.0,0.0,"613,71",24577.0,...,429.716226,0.0,0.0,34.0,33.796767,33.796767,0.499095,153.714039,153.714039,75.615274
1,53514b4c4150ad897d82dd7d42cfc1a5,2013-12-01,0.0,ЭЦН,НЕФ,РАБ.,741.0,0.0,"547,31",22893.0,...,491.506909,0.0,0.0,29.6,33.677317,33.677317,0.948523,147.318018,147.318018,65.212339
2,53514b4c4150ad897d82dd7d42cfc1a5,2014-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,544.934423,0.0,0.0,28.0,40.109561,40.109561,0.062910,170.376031,170.376031,74.861917
3,53514b4c4150ad897d82dd7d42cfc1a5,2012-12-31,0.0,ЭЦН,НЕФ,РАБ.,696.0,0.0,"554,92",23799.0,...,435.315305,0.0,0.0,34.0,31.622997,31.622997,0.379405,143.283728,143.283728,68.380790
4,53514b4c4150ad897d82dd7d42cfc1a5,2014-01-01,0.0,ЭЦН,НЕФ,РАБ.,672.0,0.0,"510,64",21172.0,...,490.294664,0.0,0.0,29.4,33.629449,33.629449,0.355954,147.212969,147.212969,64.587434
5,53514b4c4150ad897d82dd7d42cfc1a5,2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,572.880300,0.0,0.0,28.0,42.051743,42.051743,0.031061,176.133258,176.133258,80.520195
6,fe017a547ac58cc7366a60cc6bcd3915,2013-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,282.847200,0.0,0.0,25.7,101.963632,101.963632,12.802601,203.642833,203.642833,72.071408
7,fe017a547ac58cc7366a60cc6bcd3915,2014-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,460.615290,0.0,0.0,23.0,129.996230,129.996230,17.746032,159.667020,159.667020,33.234185
8,fe017a547ac58cc7366a60cc6bcd3915,2014-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,284.698161,0.0,0.0,25.4,102.768784,102.768784,13.073835,143.446751,143.446751,36.431124
9,fe017a547ac58cc7366a60cc6bcd3915,2015-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,487.273825,0.0,0.0,23.0,138.270483,138.270483,17.185420,167.444663,167.444663,30.637634


In [21]:
ids = ["Скважина"]
text = ["Причина простоя"," Куст","Состояние на конец месяца", "Причина простоя.1", "Примечание", "Мероприятия", ]
categorical = ["ГТМ", "Метод","Характер работы", "Состояние", "Агент закачки", "Пласт МЭР", "Способ эксплуатации", "Тип насоса", 
              "Состояние на конец месяца", "Фирма ГРП", "Номер бригады", "Фонтан через насос", "Нерентабельная" "Неустановившийся режим", "Назначение по проекту",
              "Группа фонда","Тип дополнительного оборудования","Тип газосепаратора", "Марка ПЭД","Тип ГЗУ","ДНС","КНС",
              "Нерентабельная"]
dates = ["Дата", "Дата ГРП","Время до псевдоуст-ся режима", "Дата запуска после КРС", "Дата пуска", "Дата останова","Дата ввода в эксплуатацию", ""]

In [22]:
combined = set(ids+text+categorical+dates)
all_cmns = set(train_main.columns)
continious = all_cmns - combined
continious

{'I X/X',
 'JD факт',
 'SKIN',
 'Буферное давление',
 'Время в накоплении',
 'Время в работе',
 'Время накопления',
 'Время работы, ч',
 'Высота перфорации',
 'Вязкость воды в пластовых условиях',
 'Вязкость жидкости в пласт. условиях',
 'Вязкость нефти в пластовых условиях',
 'ГП - Забойное давление',
 'ГП - Общий прирост Qн',
 'ГП(ГРП) Дебит жидкости',
 'ГП(ГРП) Дебит жидкости скорр-ый',
 'ГП(ИДН) Дебит жидкости',
 'ГП(ИДН) Дебит жидкости скорр-ый',
 'ГП(ИДН) Прирост дефита нефти',
 'Газ из газовой шапки, м3',
 'Газовый фактор',
 'Глубина верхних дыр перфорации',
 'Глубина спуска',
 'Глубина спуска доп. оборудования',
 'Глубина спуска.1',
 'Глубина текущего забоя',
 'Давление в линии',
 'Давление на приеме',
 'Давление наcыщения',
 'Дебит жидкости',
 'Дебит конденсата',
 'Дебит попутного газа, м3/сут',
 'Диаметр НКТ',
 'Диаметр дополнительного оборудования',
 'Диаметр плунжера',
 'Диаметр штуцера',
 'Диаметр экспл.колонны',
 'Динамическая высота',
 'Длина хода плунжера ШГН',
 'Добыча

In [23]:
train_main["Скважина"].nunique()

564

In [60]:
train_main[train_main["Нефть, т"]==0.0]

,Скважина,Дата,ГТМ,Метод,Характер работы,Состояние,"Время работы, ч",Время накопления,"Нефть, т","Попутный газ, м3",...,Характеристический дебит жидкости,Время в работе,Время в накоплении,ГП - Забойное давление,ГП(ИДН) Дебит жидкости,ГП(ИДН) Дебит жидкости скорр-ый,ГП(ИДН) Прирост дефита нефти,ГП(ГРП) Дебит жидкости,ГП(ГРП) Дебит жидкости скорр-ый,ГП - Общий прирост Qн


In [58]:
test_main.head()

,Скважина,Дата,ГТМ,Метод,Характер работы,Состояние,"Время работы, ч",Время накопления,"Попутный газ, м3","Закачка, м3",...,Пластовое давление начальное,Характеристический дебит жидкости,Время в работе,Время в накоплении,ГП - Забойное давление,ГП(ИДН) Дебит жидкости,ГП(ИДН) Дебит жидкости скорр-ый,ГП(ИДН) Прирост дефита нефти,ГП(ГРП) Дебит жидкости,ГП(ГРП) Дебит жидкости скорр-ый
0,002ff5b8a6dc271f58581e1b4fa2c5fc,2016-12-01,1.0,ФОН,НЕФ,ОСВ ТГ,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,008d0347e572a5d938a9c40c29e539fc,2013-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,255.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,00b40cb7bb8c9fd1ac26b4cc86f2b291,2018-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,277.0,521.430143,0.0,0.0,32.0,201.400991,201.400991,29.611419,0.0,0.0
3,01ba18d8b6d29875a18d4bca4eb201d7,2014-05-01,0.0,ЭЦН/ФОН,НЕФ,РАБ.,120.0,0.0,6929.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,024ec6f6e3f9c5150ecf525bf8b7a6a3,2017-06-01,1.0,ФОН,НЕФ,ОСВ ТГ,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
test_col_set = set(test_main.columns)
train_col_set = set(train_main.columns)
train_col_set - test_col_set

{'ГП - Общий прирост Qн',
 'Дебит жидкости',
 'Жидкость, м3',
 'Нефть, м3',
 'Нефть, т',
 'ТП(ГРП) Дебит жидкости',
 'ТП(ГРП) Дебит жидкости скорр-ый',
 'ТП(ИДН) Дебит жидкости',
 'ТП(ИДН) Дебит жидкости скорр-ый'}

In [1]:
train_main["Нефть, т"]

NameError: name 'train_main' is not defined

In [4]:
len(test_main)

266

In [5]:
266*6

1596